<a href="https://colab.research.google.com/github/sumalatha456/MINI_NLP-PROJECTS/blob/main/COVID_FAKE_NEWS_CLASSIFICATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
#import libraries
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle
from sklearn.linear_model import LogisticRegressionCV
import re
import pandas as pd
import warnings
warnings.filterwarnings("ignore")


In [52]:
df = pd.read_csv('covid_fake.csv')

In [53]:
df.head()

,title,text,source,label
0,Due to the recent outbreak for the Coronavirus...,"You just need to add water, and the drugs and ...",coronavirusmedicalkit.com,Fake
1,NaN,Hydroxychloroquine has been shown to have a 10...,RudyGiuliani,Fake
2,NaN,Fact: Hydroxychloroquine has been shown to hav...,CharlieKirk,Fake
3,NaN,The Corona virus is a man made virus created i...,JoanneWrightForCongress,Fake
4,NaN,Doesn’t @BillGates finance research at the Wuh...,JoanneWrightForCongress,Fake


In [54]:
df.shape

(1164, 4)

In [55]:
df ['label'].value_counts()

,count
label,
TRUE,584
Fake,345
fake,230


In [56]:
df.isnull().sum()

,0
title,82
text,10
source,20
label,5


In [57]:
df.loc[5:15]

,title,text,source,label
5,CORONA UNMASKED: Chinese Intelligence Officer ...,NaN,NaN,NaN
6,NaN,Urgent: Health Bulletin to the Public. Ministr...,Ministry of Health,Fake
7,NaN,"Pls tell ur families, relatives and friendsMOH...",NWLLAB,Fake
8,NaN,SERIOUS EXCELLENT ADVICE by Japanese doctors t...,Japanese doctors treating COVID-19 cases,Fake
9,Basic protective measures against the new coro...,Stay aware of the latest information on the CO...,https://www.who.int/emergencies/diseases/novel...,TRUE
10,NaN,The new Coronavirus may not show signs of infe...,Taiwan Experts,Fake
11,NaN,A vaccine meant for cattle can be used to figh...,facebook,Fake
12,NaN,Using a hair dryer to breathe in hot air can c...,Youtube,Fake
13,NaN,Corona virus before it reaches the lungs it re...,twitter,Fake
14,Exposing yourself to the sun or to temperature...,"You can catch COVID-19, no matter how sunny or...",https://www.who.int/emergencies/diseases/novel...,TRUE


In [58]:
df.isna().sum()

,0
title,82
text,10
source,20
label,5


In [59]:
df.loc[df['label'] == 'Fake', ['label']] = 'FAKE'
df.loc[df['label'] == 'fake', ['label']] = 'FAKE'
df.loc[df['source'] =='facbook', ['source']] = 'Facebook'
df.loc[df['label'] == 'TRUE', ['label']] = 'REAL'
df.text.fillna(df.title, inplace=True)
df.loc[5]['label'] = 'FAKE'
df.loc[15]['label'] = 'TRUE'
df.loc[43]['label'] = 'FAKE'
df.loc[131]['label'] = 'TRUE'
df.loc[245]['label'] = 'FAKE'
df.title.fillna('missing',inplace=True)
df.source.fillna('missing',inplace=True)
df.text.fillna('missing',inplace=True)
df['title_text'] = df['title'] + ' '+ df['text']

In [60]:
df.isna().sum()

,0
title,0
text,0
source,0
label,1
title_text,0


In [61]:
df.head()

,title,text,source,label,title_text
0,Due to the recent outbreak for the Coronavirus...,"You just need to add water, and the drugs and ...",coronavirusmedicalkit.com,FAKE,Due to the recent outbreak for the Coronavirus...
1,missing,Hydroxychloroquine has been shown to have a 10...,RudyGiuliani,FAKE,missing Hydroxychloroquine has been shown to h...
2,missing,Fact: Hydroxychloroquine has been shown to hav...,CharlieKirk,FAKE,missing Fact: Hydroxychloroquine has been show...
3,missing,The Corona virus is a man made virus created i...,JoanneWrightForCongress,FAKE,missing The Corona virus is a man made virus c...
4,missing,Doesn’t @BillGates finance research at the Wuh...,JoanneWrightForCongress,FAKE,missing Doesn’t @BillGates finance research at...


In [62]:
df['label'].value_counts()

,count
label,
REAL,584
FAKE,577
TRUE,2


In [63]:
df.shape

(1164, 5)

In [64]:
df.loc[5:15]

,title,text,source,label,title_text
5,CORONA UNMASKED: Chinese Intelligence Officer ...,CORONA UNMASKED: Chinese Intelligence Officer ...,missing,FAKE,CORONA UNMASKED: Chinese Intelligence Officer ...
6,missing,Urgent: Health Bulletin to the Public. Ministr...,Ministry of Health,FAKE,missing Urgent: Health Bulletin to the Public....
7,missing,"Pls tell ur families, relatives and friendsMOH...",NWLLAB,FAKE,"missing Pls tell ur families, relatives and fr..."
8,missing,SERIOUS EXCELLENT ADVICE by Japanese doctors t...,Japanese doctors treating COVID-19 cases,FAKE,missing SERIOUS EXCELLENT ADVICE by Japanese d...
9,Basic protective measures against the new coro...,Stay aware of the latest information on the CO...,https://www.who.int/emergencies/diseases/novel...,REAL,Basic protective measures against the new coro...
10,missing,The new Coronavirus may not show signs of infe...,Taiwan Experts,FAKE,missing The new Coronavirus may not show signs...
11,missing,A vaccine meant for cattle can be used to figh...,facebook,FAKE,missing A vaccine meant for cattle can be used...
12,missing,Using a hair dryer to breathe in hot air can c...,Youtube,FAKE,missing Using a hair dryer to breathe in hot a...
13,missing,Corona virus before it reaches the lungs it re...,twitter,FAKE,missing Corona virus before it reaches the lun...
14,Exposing yourself to the sun or to temperature...,"You can catch COVID-19, no matter how sunny or...",https://www.who.int/emergencies/diseases/novel...,REAL,Exposing yourself to the sun or to temperature...


In [65]:
df.isna().sum()

,0
title,0
text,0
source,0
label,1
title_text,0


#PREPROCESSING

In [66]:
df['title_text'][5]

'CORONA UNMASKED: Chinese Intelligence Officer Reveals True Magnitude of China’s Fake “Coronavirus” Crisis CORONA UNMASKED: Chinese Intelligence Officer Reveals True Magnitude of China’s Fake “Coronavirus” Crisis'

In [72]:
def preprocessor(text):
  text = re.sub('[^>]*>', '', text)
  text = re.sub(r'[^\w\s]','', text)
  text = re.sub(r'[\n]', '', text)
  text = text.lower()
  return text
df['title_text'] = df['title_text'].apply(preprocessor)
df['title_text'][5]

'corona unmasked chinese intelligence officer reveals true magnitude of chinas fake coronavirus crisis corona unmasked chinese intelligence officer reveals true magnitude of chinas fake coronavirus crisis'

In [74]:
porter = PorterStemmer()
def tokenizer(text):
  return text.split()
def tokenizer_porter(text):
  return [porter.stem(word) for word in text.split()]

TF-IDF VECTORIZER

In [78]:
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None,
                        tokenizer=tokenizer_porter,
                        use_idf=True,
                        norm='l2',
                        smooth_idf=True)
X = tfidf.fit_transform(df['title_text'])
y = df.label.values


In [79]:
X.shape

(1164, 26966)

In [81]:
#Tfdifvectorizer?? this function will give help fo any queries about the tfidf function and handle the errors.......


In [95]:
# Check for NaN values in the target variable
print(df['label'].isnull().sum())

# Impute or remove NaN values from 'label' column before splitting data
# Option 1: Impute with the most frequent value
df['label'] = df['label'].fillna(df['label'].mode()[0])

# Option 2: Remove rows with NaN values in 'label'
df = df.dropna(subset=['label'])

# Option 3: Impute based on specific logic

# After handling NaNs, re-create X and y and split the data
X = tfidf.fit_transform(df['title_text'])
y = df.label.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, shuffle=False)


0


In [96]:

# Proceed with model fitting
clf = LogisticRegressionCV(cv=5,
                           scoring='accuracy',
                           random_state=0,
                           n_jobs=-1,
                           verbose=0,
                           max_iter=300).fit(X_train, y_train)

In [98]:
fake_news_model = open('fake_news_model.pkl','wb')
pickle.dump(clf, fake_news_model)
fake_news_model.close()

In [100]:
#model evaluation
filename = 'fake_news_model.pkl'  # Changed filename to match the saved file
saved_clf = pickle.load(open(filename, 'rb'))
saved_clf.score(X_test, y_test)

0.9257142857142857

In [101]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred = saved_clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[119  13   0]
 [ 12 205   1]
 [  0   0   0]]
              precision    recall  f1-score   support

        FAKE       0.91      0.90      0.90       132
        REAL       0.94      0.94      0.94       218
        TRUE       0.00      0.00      0.00         0

    accuracy                           0.93       350
   macro avg       0.62      0.61      0.62       350
weighted avg       0.93      0.93      0.93       350



In [105]:
#sample prediction
clf.predict(X_test[59])

array(['FAKE'], dtype=object)

In [106]:
test = "Corona virus before it reaches the lungs"
inp = [test]
vect = tfidf.transform(inp)
prediction = clf.predict(vect)
print(prediction)

['FAKE']
